In [18]:
open System

// Types
type OptionType = Call | Put
type OptionParams = {
    S: float          // Current stock price
    K: float          // Strike price
    T: float          // Time to expiration (in years)
    R: float          // Risk-free rate
    Q: float          // Dividend yield
    InitialVol: float // Initial volatility guess
    N: int            // Number of steps in the binomial tree
    VMarket: float    // Market price of the option
    OptionType: OptionType
}

type GreeksResult = {
    Delta: float
    Theta: float
    Gamma: float
}

type CalculationResult = {
    ImpliedVol: float
    Iterations: int
    CalculatedPrice: float
    Greeks: GreeksResult
}

// Helper functions
let calculateOptionPriceAndGreeks (params: OptionParams) (sigma: float) =
    let dt = params.T / float params.N
    let u = exp (sigma * sqrt dt)
    let d = 1.0 / u
    let p = (exp ((params.R - params.Q) * dt) - d) / (u - d)
    
    // Generate stock price tree
    let generatePrices i j = params.S * (pown u j) * (pown d (i-j))
    let stockPrices = Array2D.init (params.N + 1) (params.N + 1) generatePrices
    
    // Initialize option value at expiration
    let optionValue = Array2D.create (params.N + 1) (params.N + 1) 0.0
    for j in 0..params.N do
        optionValue.[params.N, j] <- 
            match params.OptionType with
            | Call -> max 0.0 (stockPrices.[params.N, j] - params.K)
            | Put -> max 0.0 (params.K - stockPrices.[params.N, j])
    
    // Backward induction
    for i in (params.N - 1) .. -1 .. 0 do
        for j in 0..i do
            let continuationValue = exp(-params.R * dt) * (p * optionValue.[i+1, j+1] + (1.0-p) * optionValue.[i+1, j])
            let exerciseValue = 
                match params.OptionType with
                | Call -> max 0.0 (stockPrices.[i, j] - params.K)
                | Put -> max 0.0 (params.K - stockPrices.[i, j])
            optionValue.[i, j] <- max continuationValue exerciseValue
    
    // Calculate Greeks
    let delta = (optionValue.[1, 1] - optionValue.[1, 0]) / (stockPrices.[1, 1] - stockPrices.[1, 0])
    
    let theta = (optionValue.[2, 1] - optionValue.[0, 0]) / (2.0 * dt)
    
    let gamma = ((optionValue.[2, 2] - optionValue.[2, 1]) / (stockPrices.[2, 2] - stockPrices.[2, 1]) -
                 (optionValue.[2, 1] - optionValue.[2, 0]) / (stockPrices.[2, 1] - stockPrices.[2, 0])) /
                ((stockPrices.[2, 2] - stockPrices.[2, 0]) / 2.0)
    
    optionValue.[0, 0], { Delta = delta; Theta = theta; Gamma = gamma }

let vega (params: OptionParams) (sigma: float) =
    let delta = 0.0001
    let priceUp, _ = calculateOptionPriceAndGreeks params (sigma + delta)
    let priceDown, _ = calculateOptionPriceAndGreeks params (sigma - delta)
    (priceUp - priceDown) / (2.0 * delta)

// Main algorithm
let calculateImpliedVolatilityAndGreeks (params: OptionParams) =
    let tolerance = 0.0001
    let maxIterations = 100
    
    let rec newtonRaphson sigma iteration =
        if iteration >= maxIterations then
            None
        else
            let calculatedPrice, greeks = calculateOptionPriceAndGreeks params sigma
            if abs (calculatedPrice - params.VMarket) < tolerance then
                Some (sigma, iteration, calculatedPrice, greeks)
            else
                let vegaValue = vega params sigma
                if vegaValue = 0.0 then
                    None
                else
                    let newSigma = sigma - (calculatedPrice - params.VMarket) / vegaValue
                    let adjustedSigma = max 0.0001 newSigma
                    newtonRaphson adjustedSigma (iteration + 1)
    
    match newtonRaphson params.InitialVol 0 with
    | Some (impliedVol, iterations, calculatedPrice, greeks) ->
        Ok { ImpliedVol = impliedVol; Iterations = iterations; CalculatedPrice = calculatedPrice; Greeks = greeks }
    | None ->
        Error "Implied volatility calculation did not converge"


//  S: float          // Current stock price
//     K: float          // Strike price
//     T: float          // Time to expiration (in years)
//     R: float          // Risk-free rate
//     Q: float          // Dividend yield
//     InitialVol: float // Initial volatility guess
//     N: int            // Number of steps in the binomial tree
//     VMarket: float    // Market price of the option
//     OptionType: OptionType

let currentDate = DateTime.Parse("2020-11-28")
let expirationDate = DateTime.Parse("2021-01-15")

let timeToExpiration = (expirationDate - currentDate).TotalDays / 365.0

printfn "Time to expiration: %.4f" timeToExpiration

let params = {
    S = 215.29 // current spot price
    K = 215.0  // strike price
    T = timeToExpiration 
    R = 0.0009  // risk-free rate
    Q = 0.0105  // dividend yield
    InitialVol = 0.1 // initial volatility guess
    N = 100   // number of steps in the binomial tree
    VMarket = 8.7  // market price of the option
    OptionType = Call 
}

match calculateImpliedVolatilityAndGreeks params with
| Ok result ->
    printfn "Implied Volatility: %.4f" result.ImpliedVol
    printfn "Iterations: %d" result.Iterations
    printfn "Calculated Price: %.4f" result.CalculatedPrice
    printfn "Delta: %.4f" result.Greeks.Delta
    printfn "Theta: %.4f" (result.Greeks.Theta / 365.0)
    printfn "Gamma: %.4f" result.Greeks.Gamma
| Error msg ->
    printfn "Error: %s" msg

Time to expiration: 0.1315
Implied Volatility: 0.2794
Iterations: 2
Calculated Price: 8.7000
Delta: 0.5210
Theta: -0.0883
Gamma: 0.0184


In [27]:
#r "nuget: MathNet.Numerics.FSharp"
#r "nuget: MathNet.Numerics"

open MathNet.Numerics
open System


// Normal cumulative distribution function
let normalCdf x =
    0.5 * (1.0 + SpecialFunctions.Erf(x / Math.Sqrt(2.0)))

// Define the normal probability density function
let normalPdf x =
    Math.Exp(-0.5 * x * x) / Math.Sqrt(2.0 * Math.PI)

// Black-Scholes formula for call option price
let blackScholesCallPrice s k t r v =
    let d1 = (Math.Log(s / k) + (r + 0.5 * v * v) * t) / (v * Math.Sqrt(t))
    let d2 = d1 - v * Math.Sqrt(t)
    s * normalCdf d1 - k * Math.Exp(-r * t) * normalCdf d2

// Black-Scholes formula for put option price
let blackScholesPutPrice s k t r v =
    let d1 = (Math.Log(s / k) + (r + 0.5 * v * v) * t) / (v * Math.Sqrt(t))
    let d2 = d1 - v * Math.Sqrt(t)
    k * Math.Exp(-r * t) * normalCdf -d2 - s * normalCdf -d1

// Delta calculation
let delta s k t r v optionType =
    let d1 = (Math.Log(s / k) + (r + 0.5 * v * v) * t) / (v * Math.Sqrt(t))
    match optionType with
    | "call" -> normalCdf d1
    | "put" -> normalCdf d1 - 1.0
    | _ -> failwith "Invalid option type"

// Theta calculation
let theta s k t r v optionType =
    let d1 = (Math.Log(s / k) + (r + 0.5 * v * v) * t) / (v * Math.Sqrt(t))
    let d2 = d1 - v * Math.Sqrt(t)
    let term1 = -(s * normalPdf d1 * v) / (2.0 * Math.Sqrt(t))
    let term2 = r * k * Math.Exp(-r * t) * normalCdf d2
    match optionType with
    | "call" -> term1 - term2 + r * s * normalCdf d1
    | "put" -> term1 + term2 - r * s * normalCdf -d1
    | _ -> failwith "Invalid option type"

// Gamma calculation
let gamma s k t r v =
    let d1 = (Math.Log(s / k) + (r + 0.5 * v * v) * t) / (v * Math.Sqrt(t))
    normalPdf d1 / (s * v * Math.Sqrt(t))

// Vega calculation
let vega s k t r v =
    let d1 = (Math.Log(s / k) + (r + 0.5 * v * v) * t) / (v * Math.Sqrt(t))
    s * Math.Sqrt(t) * normalPdf d1

// Rho calculation
let rho s k t r v optionType =
    let d2 = (Math.Log(s / k) + (r + 0.5 * v * v) * t) / (v * Math.Sqrt(t)) - v * Math.Sqrt(t)
    match optionType with
    | "call" -> k * t * Math.Exp(-r * t) * normalCdf d2
    | "put" -> -k * t * Math.Exp(-r * t) * normalCdf -d2
    | _ -> failwith "Invalid option type"

// Newton-Raphson method to find implied volatility
let impliedVolatility targetPrice s k t r optionType =
    let epsilon = 1e-5
    let rec newtonRaphson (v:float) guess =
        if Math.Abs(v) < epsilon then guess
        else
            let price =
                match optionType with
                | "call" -> blackScholesCallPrice s k t r guess
                | "put" -> blackScholesPutPrice s k t r guess
                | _ -> failwith "Invalid option type"
            let vega = s * Math.Sqrt(t) * normalPdf ((Math.Log(s / k) + (r + 0.5 * guess * guess) * t) / (guess * Math.Sqrt(t)))
            let nextGuess = guess - (price - targetPrice) / vega
            newtonRaphson (price - targetPrice) nextGuess
    newtonRaphson (targetPrice - match optionType with | "call" -> blackScholesCallPrice s k t r 0.1 | "put" -> blackScholesPutPrice s k t r 0.1 | _ -> failwith "Invalid option type") 0.1

// Sample input/output example
let spotPrice = 74.64
let strikePrice = 75.0
let timeToMaturity = 0.129
let riskFreeRate = 0.0009
let marketPrice = 4.35 // observed market price of the call option

let optionType = "call"

let iv = impliedVolatility marketPrice spotPrice strikePrice timeToMaturity riskFreeRate optionType
printfn "The implied volatility is: %f" iv

let deltaValue = delta spotPrice strikePrice timeToMaturity riskFreeRate iv optionType
let thetaValue = theta spotPrice strikePrice timeToMaturity riskFreeRate iv optionType
let gammaValue = gamma spotPrice strikePrice timeToMaturity riskFreeRate iv
let vegaValue = vega spotPrice strikePrice timeToMaturity riskFreeRate iv
let rhoValue = rho spotPrice strikePrice timeToMaturity riskFreeRate iv optionType

printfn "Delta: %f" deltaValue
printfn "Theta: %f" (thetaValue / 365.0)
printfn "Gamma: %f" gammaValue
printfn "Vega: %f" (vegaValue / 100.0)
printfn "Rho: %f" (rhoValue / 100.0)




Installed Packages MathNet.Numerics, 5.0.0 MathNet.Numerics.FSharp, 5.0.0

The implied volatility is: 0.422369
Delta: 0.517906
Theta: -0.047909
Gamma: 0.035198
Vega: 0.106841
Rho: 0.044255
